In [1]:
import numpy as np
from PIL import Image
import csv, os
import random

data_file_path = "/data/faces_images"
label_file_path = "/data/_newtrain_vision.csv"

f = open('/data/new_train_vision.csv','w')
wr = csv.writer(f)
wr.writerow(['filename','label'])
with open(label_file_path) as path_list:
    for data in enumerate(path_list):
        if data[0] is 0: continue
        filename, label = data[1].split(",")
        filepath = os.path.join(data_file_path,filename)
        #img = Image.open(filepath)
        wr.writerow([filename, int(label)])
        for i in range(1,4):
            #img = img.rotate(90)
            #img.save(os.path.join(data_file_path,'rot'+str(90*i)+'_'+filename))
            wr.writerow(['rot'+str(90*i)+'_'+filename, int(label)])
                      

IOError: [Errno 2] No such file or directory: '/data/_newtrain_vision.csv'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os, sys
import numpy as np
import preprocess.image as image
import tensorflow.keras.utils as utils
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import GeneratorEnqueuer
from tensorflow.keras.utils import OrderedEnqueuer
print (tf.__version__)
import random

data_file_path = "/data/faces_images"
label_file_path = "/data/train_vision.csv"
initial_learning_rate = 0.001
image_size = 299
class_num = 6
batch_size = 8
epoch = 20


filenames = []
labels = []
with open(label_file_path) as path_list:
    for data in enumerate(path_list):
        if data[0] is 0: continue
        filename, label = data[1].split(",")
        filenames.append(os.path.join(data_file_path,filename))
        labels.append(int(label)-1)
    #random.seed(1)    
    #random.shuffle(filenames)
    #random.shuffle(labels)
    print('Found %d images for training.' % (len(filenames)))
    
class_num = list(range(class_num))
train_datagen = image.ImageDataGenerator(rescale=1. / 255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True)
    
train_generator = train_datagen.flow_from_mapfile(class_num,filenames,labels,
                                            target_size=(image_size,image_size),
                                            batch_size=batch_size,
                                            class_mode='categorical')

2.0.0
Found 5850 images for training.


In [ ]:
import net.nets_factory as nets_factory
from tensorflow.keras import optimizers

network_name = 'inception_resnet_v2'
save_model_path = "/home/june/model"
network_fn = nets_factory.get_network(network_name)
model = network_fn(include_top=True, weights=None, classes=len(class_num))


lr_schedule = optimizers.schedules.ExponentialDecay(initial_learning_rate,
                                                    decay_steps=1000,
                                                    decay_rate=0.9,
                                                    staircase=False)
#optimizer = optimizers.SGD(learning_rate = lr_schedule)
#optimizer = optimizers.RMSprop(learning_rate = lr_schedule)
optimizer = optimizers.Adagrad(learning_rate = lr_schedule)
model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])

#model.load_weights(os.path.join(save_model_path,"inception_v3_10.ckpt"))

def create_data_queue(train_generator):
    is_sequence = isinstance(train_generator, Sequence)
    wait_time = 0.01
    if is_sequence:
        enqueuer = OrderedEnqueuer(train_generator,
                                use_multiprocessing=False,
                                shuffle = False)
    else:
        enqueuer = GeneratorEnqueuer(train_generator,
                                use_multiprocessing =False,
                                wait_time = wait_time)
    return enqueuer



enqueuer = create_data_queue(train_generator)
enqueuer.start(workers = 1, max_queue_size=10)
output_generator = enqueuer.get()
accuracy_mean = 0.
current_step = 0
current_epoch = 1
max_iter = len(filenames) / batch_size

try:
    while current_step < epoch * max_iter:
        generator_output = next(output_generator)
        x, y = generator_output
        loss, accuracy = model.train_on_batch(x,y)            
        accuracy_mean = ((accuracy_mean * current_step)+accuracy) / (current_step+1)
                
        sys.stdout.write('\r>> epoch : %d, loss : %f, accuracy : %f, step : %d/%d ' % (int(current_step/len(filenames)), loss, accuracy, current_step, max_iter))
        sys.stdout.flush()
        current_step += 1

except Exception as ex:
    print(ex)
    enqueuer.stop(timeout=0.01)
    exit()
    
enqueuer.stop(timeout=0.01)
model.save_weights(os.path.join(save_model_path,'%s_%d_aug.ckpt'%(network_name,epoch)))

>> epoch : 1, loss : 0.139122, accuracy : 1.000000, step : 9445/731 

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

In [1]:
test_file_path = "/data/test_vision.csv"
import net.nets_factory as nets_factory
from tensorflow.keras.preprocessing import image as kimage
import csv, os
import numpy as np
from tensorflow.keras import optimizers

def load_image(image_file_path):
        img = kimage.load_img(image_file_path, target_size=(image_size, image_size))
        x = kimage.img_to_array(img)
        x /= 255.
        return x
    
result = []
data_file_path = "/data/faces_images"
save_model_path = "/home/june/model"
network_fn = nets_factory.get_network('inception_v3')
model = network_fn(include_top=True, weights=None, classes=6)
image_size = 299

lr_schedule = optimizers.schedules.ExponentialDecay(0.001,
                                                    decay_steps=1000,
                                                    decay_rate=0.9,
                                                    staircase=False)

optimizer = optimizers.SGD(learning_rate = lr_schedule)
model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])



model.load_weights(os.path.join(save_model_path,"inception_v3_30.ckpt"))

             
with open(test_file_path) as path_list:
    for image_path in enumerate(path_list):
        if image_path[0] is 0 : continue
        img = load_image(os.path.join(data_file_path,image_path[1].split("\n")[0]))
        x = np.expand_dims(img, axis=0)
        softmax = model.predict(x, batch_size=1)
        result.append(np.asarray(softmax).argmax() + 1)
        
#df = pd.DataFrame(result, columns=['label'])
#df.to_csv("/data/result.csv")
f = open('/data/result_rms.csv','w')
wr = csv.writer(f)
wr.writerow(['prediction'])
for i in result:
    wr.writerow([i])
f.close()